# Load a genome to the HealthOmics Sequence Store
---
This notebook demonstrates how to download a public genome to a HealthOmics sequence store. This is 
in preparation for later pretraining of a genomic language model (HyenaDNA).

## Prerequisites

1. In order to download and process the data you should make sure that this notebook has access to at least
64Gb of disc storage.
2. The notebook needs to have permissions to access S3. To add these permissions, follow [these instructions](https://docs.aws.amazon.com/omics/latest/dev/manage-reference-store.html). You
can see what your notebook's execution role by running this: `print(sagemaker.get_execution_role())`

In [25]:
from pathlib import Path
import importlib
from functools import partial
from time import sleep

import utilities as u
# to reload the utilities without restarting the kernel, use this: importlib.reload(u)

import boto3
import sagemaker

In [26]:
sequence_store_name = "mouse genome"
# This is where we upload the compressed FASTQ files:
bucket_name = "sgh-misc"
prefix = "data/mouse/"
import_job_role_arn = "..."
kms_arn = "..."

In [27]:
omics = boto3.client("omics")
s3 = boto3.client("s3")

First, we download a [mouse reference genome](https://www.ncbi.nlm.nih.gov/datasets/genome/GCA_921998355.2/)
from Genbank onto the local disk (this should take less than a minute with a broadband connection). There should
be one (compressed) FASTA file per chromosome.

In [ ]:
!wget -P mouse/ -r -nH --cut-dirs=11 --no-parent ftp://ftp.ncbi.nlm.nih.gov/genomes/genbank/vertebrate_mammalian/Mus_musculus/latest_assembly_versions/GCA_921998355.2_A_J_v3/GCA_921998355.2_A_J_v3_assembly_structure/Primary_Assembly/assembled_chromosomes/FASTA/

If the above command finishes successfully then you should be able to verify the results
and see something similar to this:
```bash
sh-4.2$ ls -l ~/SageMaker/mouse
total 703768
-rw-rw-r-- 1 ec2-user ec2-user 36522179 Jul 19  2022 chr10.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 34213890 Jul 19  2022 chr11.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 32615588 Jul 19  2022 chr12.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 32962659 Jul 19  2022 chr13.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 32646782 Jul 19  2022 chr14.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 29093156 Jul 19  2022 chr15.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 27225255 Jul 19  2022 chr16.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 26229898 Jul 19  2022 chr17.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 25313610 Jul 19  2022 chr18.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 16757414 Jul 19  2022 chr19.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 55286326 Jul 19  2022 chr1.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 50752623 Jul 19  2022 chr2.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 44959307 Jul 19  2022 chr3.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 42502452 Jul 19  2022 chr4.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 41498510 Jul 19  2022 chr5.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 41907927 Jul 19  2022 chr6.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 38157680 Jul 19  2022 chr7.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 35790409 Jul 19  2022 chr8.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 34764818 Jul 19  2022 chr9.fna.gz
-rw-rw-r-- 1 ec2-user ec2-user 41422059 Jul 19  2022 chrX.fna.gz
```

In [28]:
data = Path.home() / "SageMaker" / "mouse"

Next, we uncompress the compressed FASTA (".fna.gz") files to create FASTA (".fna") files:

In [29]:
fasta_files = u.convert_directory(data, suffix=".fna.gz",
                                  convertor=partial(u.gunzip_file,
                                                    suffix=".gz"))

/home/ec2-user/SageMaker/mouse/chr11.fna.gz -> /home/ec2-user/SageMaker/mouse/chr11.fna
/home/ec2-user/SageMaker/mouse/chr6.fna.gz -> /home/ec2-user/SageMaker/mouse/chr6.fna
/home/ec2-user/SageMaker/mouse/chr3.fna.gz -> /home/ec2-user/SageMaker/mouse/chr3.fna
/home/ec2-user/SageMaker/mouse/chr1.fna.gz -> /home/ec2-user/SageMaker/mouse/chr1.fna
/home/ec2-user/SageMaker/mouse/chr8.fna.gz -> /home/ec2-user/SageMaker/mouse/chr8.fna
/home/ec2-user/SageMaker/mouse/chrX.fna.gz -> /home/ec2-user/SageMaker/mouse/chrX.fna
/home/ec2-user/SageMaker/mouse/chr7.fna.gz -> /home/ec2-user/SageMaker/mouse/chr7.fna
/home/ec2-user/SageMaker/mouse/chr14.fna.gz -> /home/ec2-user/SageMaker/mouse/chr14.fna
/home/ec2-user/SageMaker/mouse/chr2.fna.gz -> /home/ec2-user/SageMaker/mouse/chr2.fna
/home/ec2-user/SageMaker/mouse/chr15.fna.gz -> /home/ec2-user/SageMaker/mouse/chr15.fna
/home/ec2-user/SageMaker/mouse/chr12.fna.gz -> /home/ec2-user/SageMaker/mouse/chr12.fna
/home/ec2-user/SageMaker/mouse/chr5.fna.gz -> 

Now, we convert those FASTA files into FASTQ files:

In [30]:
fastq_files = u.convert_directory(data, suffix=".fna",
                                  convertor=u.convert_fasta_file_to_fastq)

/home/ec2-user/SageMaker/mouse/chr2.fna -> /home/ec2-user/SageMaker/mouse/chr2.fq
/home/ec2-user/SageMaker/mouse/chrX.fna -> /home/ec2-user/SageMaker/mouse/chrX.fq
/home/ec2-user/SageMaker/mouse/chr6.fna -> /home/ec2-user/SageMaker/mouse/chr6.fq
/home/ec2-user/SageMaker/mouse/chr3.fna -> /home/ec2-user/SageMaker/mouse/chr3.fq
/home/ec2-user/SageMaker/mouse/chr5.fna -> /home/ec2-user/SageMaker/mouse/chr5.fq
/home/ec2-user/SageMaker/mouse/chr9.fna -> /home/ec2-user/SageMaker/mouse/chr9.fq
/home/ec2-user/SageMaker/mouse/chr10.fna -> /home/ec2-user/SageMaker/mouse/chr10.fq
/home/ec2-user/SageMaker/mouse/chr16.fna -> /home/ec2-user/SageMaker/mouse/chr16.fq
/home/ec2-user/SageMaker/mouse/chr8.fna -> /home/ec2-user/SageMaker/mouse/chr8.fq
/home/ec2-user/SageMaker/mouse/chr4.fna -> /home/ec2-user/SageMaker/mouse/chr4.fq
/home/ec2-user/SageMaker/mouse/chr15.fna -> /home/ec2-user/SageMaker/mouse/chr15.fq
/home/ec2-user/SageMaker/mouse/chr7.fna -> /home/ec2-user/SageMaker/mouse/chr7.fq
/home/ec2-

And then we compress these FASTA files:

In [31]:
compressed_fq_files = u.convert_directory(data, suffix=".fq",
                                          convertor=u.gzip_file)

/home/ec2-user/SageMaker/mouse/chr14.fq -> /home/ec2-user/SageMaker/mouse/chr14.fq.gz
/home/ec2-user/SageMaker/mouse/chr1.fq -> /home/ec2-user/SageMaker/mouse/chr1.fq.gz
/home/ec2-user/SageMaker/mouse/chr18.fq -> /home/ec2-user/SageMaker/mouse/chr18.fq.gz
/home/ec2-user/SageMaker/mouse/chr12.fq -> /home/ec2-user/SageMaker/mouse/chr12.fq.gz
/home/ec2-user/SageMaker/mouse/chr13.fq -> /home/ec2-user/SageMaker/mouse/chr13.fq.gz
/home/ec2-user/SageMaker/mouse/chr6.fq -> /home/ec2-user/SageMaker/mouse/chr6.fq.gz
/home/ec2-user/SageMaker/mouse/chr11.fq -> /home/ec2-user/SageMaker/mouse/chr11.fq.gz
/home/ec2-user/SageMaker/mouse/chr10.fq -> /home/ec2-user/SageMaker/mouse/chr10.fq.gz
/home/ec2-user/SageMaker/mouse/chr8.fq -> /home/ec2-user/SageMaker/mouse/chr8.fq.gz
/home/ec2-user/SageMaker/mouse/chrX.fq -> /home/ec2-user/SageMaker/mouse/chrX.fq.gz
/home/ec2-user/SageMaker/mouse/chr3.fq -> /home/ec2-user/SageMaker/mouse/chr3.fq.gz
/home/ec2-user/SageMaker/mouse/chr4.fq -> /home/ec2-user/SageMak

Next, we upload the files to S3

In [32]:
s3_uris = []
for file in compressed_fq_files:
    key = f"{prefix}{file.name}"
    s3.upload_file(file, bucket_name, key)
    s3_uri = f"s3://{bucket_name}/{key}"
    print(s3_uri)
    s3_uris.append(s3_uri)
print("Done")

s3://sgh-misc/data/mouse/chr14.fq.gz
s3://sgh-misc/data/mouse/chr1.fq.gz
s3://sgh-misc/data/mouse/chr18.fq.gz
s3://sgh-misc/data/mouse/chr12.fq.gz
s3://sgh-misc/data/mouse/chr13.fq.gz
s3://sgh-misc/data/mouse/chr6.fq.gz
s3://sgh-misc/data/mouse/chr11.fq.gz
s3://sgh-misc/data/mouse/chr10.fq.gz
s3://sgh-misc/data/mouse/chr8.fq.gz
s3://sgh-misc/data/mouse/chrX.fq.gz
s3://sgh-misc/data/mouse/chr3.fq.gz
s3://sgh-misc/data/mouse/chr4.fq.gz
s3://sgh-misc/data/mouse/chr9.fq.gz
s3://sgh-misc/data/mouse/chr19.fq.gz
s3://sgh-misc/data/mouse/chr5.fq.gz
s3://sgh-misc/data/mouse/chr2.fq.gz
s3://sgh-misc/data/mouse/chr16.fq.gz
s3://sgh-misc/data/mouse/chr7.fq.gz
s3://sgh-misc/data/mouse/chr17.fq.gz
s3://sgh-misc/data/mouse/chr15.fq.gz
Done


Next, we create a sequence store in HealthOmics

In [33]:
seq_store_resp = omics.create_sequence_store(
    name=sequence_store_name,
    description="GCA_921998355.2_A_J_v3",
    sseConfig={
        "type": "KMS",
        "keyArn": kms_arn
    })
seq_store_id = seq_store_resp["id"]

Next, we load our FASTQ files into this new sequence store

In [34]:
import_job_resp = omics.start_read_set_import_job(
    sequenceStoreId=seq_store_id,
    roleArn=import_job_role_arn,
    # roleArn="arn:aws:iam::111918798052:role/OmicsServiceRole-20221003T082108",
    sources=[
        {
            "sourceFiles": {"source1": s3_uri},
            "sourceFileType": "FASTQ",
            "subjectId": "N/A",
            "sampleId": "N/A",
        }
        for s3_uri in s3_uris
    ]
)
import_job_id = import_job_resp["id"]

We now wait for these read sets to be imported into the sequence store. This typically takes about an hour.

In [35]:
%%time
while True:
    job_list_response = omics.list_read_set_import_jobs(maxResults=100,
                                                        sequenceStoreId=seq_store_id)
    import_jobs = [job for job in job_list_response["importJobs"] if job["id"] == import_job_id]
    [status] = [job["status"] for job in import_jobs] # filtered on job id, so should only be one
    print(f"Status of import job {import_job_id} is {status}")
    if status not in {"SUBMITTED", "IN_PROGRESS"}:
        break
    sleep(5*60)
print("Done")

Status of import job 1483876671 is IN_PROGRESS
Status of import job 1483876671 is IN_PROGRESS
Status of import job 1483876671 is IN_PROGRESS
Status of import job 1483876671 is IN_PROGRESS
Status of import job 1483876671 is IN_PROGRESS
Status of import job 1483876671 is IN_PROGRESS
Status of import job 1483876671 is IN_PROGRESS
Status of import job 1483876671 is IN_PROGRESS
Status of import job 1483876671 is IN_PROGRESS
Status of import job 1483876671 is IN_PROGRESS
Status of import job 1483876671 is IN_PROGRESS
Status of import job 1483876671 is COMPLETED
Done
CPU times: user 195 ms, sys: 70.2 ms, total: 265 ms
Wall time: 55min 1s
